In [ ]:
!unzip tennis_court_det_dataset.zip

# Start code

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Torch Dataset

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, 'r') as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        imf = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust the x coordinates
        kps[1::2] *= 224.0 / h # Adjust the y coordinates

        return img, kps

In [ ]:
train_dataset = KeypointsDataset('data/images', 'data/data_train.json')
val_dataset = KeypointsDataset('data/images', 'data/data_val.json')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

# Create model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

In [ ]:
model = model.to(device)

# Train the model

In [ ]:
crit = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

In [ ]:
epcohs = 50
for epoch in range(epcohs):
    for i, (img,kps) in enumerate(train_loader):
        img = img.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        pred = model(img)
        loss = crit(pred, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")